In [ ]:
%matplotlib inline

In [ ]:
import atmc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
from pytpc.constants import pi, degrees
import yaml

In [ ]:
import pytpc

In [ ]:
with open('../../ar40-aug15/fitters/config_e15503b_macmini.yml') as f:
    config = yaml.load(f)
# config['beam_center'] = [0, 0, 0]
# config['micromegas_gain'] = 100

In [ ]:
efield = np.array(config['efield'])
bfield = np.array(config['bfield'])
vd = np.array(config['vd'])
clock = config['clock']
shape = float(config['shape'])
ioniz = config['ioniz']
tilt = config['tilt'] * degrees

mass_num = 1
charge_num = 1
gain = 1

gas = pytpc.gases.InterpolatedGas('isobutane', 18.)
ens = np.arange(0, 100*1000, dtype='int')
eloss = gas.energy_loss(ens / 1000, mass_num, charge_num)

In [ ]:
tracker = atmc.mcopt_wrapper.PyTracker(mass_num, charge_num, eloss, efield, bfield)

with h5py.File('/Users/josh/Documents/Code/ar40-aug15/monte_carlo/LUT.h5', 'r') as hf:
    lt = hf['LUT'][:]
padplane = atmc.mcopt_wrapper.PyPadPlane(lt, -0.280, 0.0001, -0.280, 0.0001, -108*degrees)

evtgen = atmc.mcopt_wrapper.PyEventGenerator(padplane, vd, clock * 1e6, shape, mass_num, ioniz, gain, tilt)
mcmin = atmc.mcopt_wrapper.PyMCminimizer(tracker, evtgen)

In [ ]:
tr = tracker.track_particle(0, 0, 0.832620, 4.337075, 3.652245, 2.173140)
raw_xyz = evtgen.make_peaks(tr[:, :3].copy(), tr[:, 4].copy())

In [ ]:
tr.shape

In [ ]:
plt.plot(tr[:, 2], np.hypot(tr[:, 0], tr[:, 1]))

In [ ]:
plt.plot(raw_xyz[:, 2], np.hypot(raw_xyz[:, 0], raw_xyz[:, 1]), '.')

In [ ]:
raw_xyz